In [1]:
import pandas as pd
import sys
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import pandas as pd

from striprtf.striprtf import rtf_to_text

In [100]:
sample = pd.read_csv('../data/sample.csv')
from_folders = pd.read_csv('../data/from_folders.csv')
external_data = pd.read_csv('../data/external_data.csv')
external_data['text'] = external_data['text'].str.split('Обратите внимание', expand=True)[0]

In [123]:
REPLACE_WORDS = {
    "доверенность" : re.compile(r"\bд\s*о\s*в\s*е\s*р\s*е\s*н\s*н\s*о\s*с\s*т\s*ь\b"),
    "договор" : re.compile(r"\bд\s*о\s*г\s*о\s*в\s*о\s*р\b"),
    "акт": re.compile(r"\bа\s*к\s*т\b"),
    "заявление": re.compile(r"\bз\s*а\s*я\s*в\s*л\s*е\s*н\s*и\s*е\b"),
    "приказ": re.compile(r"\bп\s*р\s*и\s*к\s*а\s*з\b"),
    "счет": re.compile(r"\bс\s*ч\s*е\s*т\b"),
    "приложение": re.compile(r"\bп\s*р\s*и\s*л\s*о\s*ж\s*е\s*н\s*и\s*e\b"),
    "соглашение": re.compile(r"\bс\s*о\s*г\s*л\s*а\s*ш\s*е\s*н\s*и\s*e\b"),
    "договор оферты": re.compile(r"\bд\s*о\s*г\s*о\s*в\s*о\s*р\s*о\s*ф\s*е\s*р\s*т\s*ы\b"),
    "устав": re.compile(r"\bу\s*с\s*т\s*а\s*в\b"),
    "решение": re.compile(r"\bр\s*е\s*ш\s*е\s*н\s*и\s*е\b")
}

MAPPING = {
    "proxy": "доверенность",
    "contract": "договор",
    "act": "акт",
    "application": "заявление",
    "order": "приказ",
    "invoice": "счет",
    "bill": "приложение",
    "arrangement": "соглашение",
    "contract offer": "договор оферты",
    "statute": "устав",
    "determination": "решение",
}

In [124]:
def clean_text(text: str) -> str:
    for key, value in REPLACE_WORDS.items():
        text = re.sub(value, key, text)

    text = re.sub(r'[^;:\?!,\.\-а-яА-Яa-zA-Z\s<]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\.\.+', '', text)
    
    text = ' '.join([word for word in text.split() if len(word) > 1])
    text = re.sub(r'\s+', ' ', text)

    return text


def processing(df: pd.DataFrame) -> pd.DataFrame:
    df_ = df.copy()
    df_['text'] = df_['text'].str.lower()
    df_['text'] = df_['text'].apply(clean_text)
    return df_

In [125]:
clear_sample = processing(sample)
clear_from_folders = processing(from_folders)
clear_external_data = processing(external_data)

In [126]:
clear_sample.head()

,class,text,clear_text
0,arrangement,соглашение расторжении трудового договора от г...,соглашение расторжении трудового договора от к...
1,arrangement,соглашение предоставлении опциона на заключени...,соглашение предоставлении опциона на заключени...
2,arrangement,соглашение реструктуризации задолженности г. и...,соглашение реструктуризации задолженности ижев...
3,arrangement,дополнительное соглашение договору купли-прода...,дополнительное соглашение договору куплипродаж...
4,arrangement,соглашение расторжении договора об оказании ус...,соглашение расторжении договора об оказании ус...


In [127]:
clear_from_folders.head()

,class,text
0,statute,утверждено на учредительной конференции предст...
1,statute,министерство обороны российской федерации кора...
2,statute,введен действие приказом министра обороны росс...
3,statute,принят учредительным съездом профсоюзов рсфср ...
4,statute,утвержден всероссийским съездом адвокатов янва...


In [128]:
clear_external_data.head()

,class,link,path,text,clear_text,clear_text_in_sample
0,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-registracziyu-ts-v-gibd...,доверенность г. ижевск дата выдачи: третье сен...,доверенность ижевск дата выдачи третье сентябр...,1
1,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-zaverenie-kopij-dokumentov,"ооо артемида г. ижевск, ул. сосновая, д. а, те...",ооо артемида ижевск ул сосновая тел огрн инн к...,1
2,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-denezhnyh-sr...,общество ограниченной ответственностью сигма г...,общество ограниченной ответственностью сигма м...,1
3,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-predstavlenie-interesov...,доверенность г. ижевск двадцать пятое сентября...,доверенность ижевск двадцать пятое сентября дв...,1
4,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-trudovoj-kni...,доверенность г. ижевск тринадцатое марта две т...,доверенность ижевск тринадцатое марта две тыся...,1


In [129]:
external_data['clear_text'] = clear_external_data['text'].copy()
sample['clear_text'] = clear_sample['text'].copy()

In [130]:
external_data['clear_text_in_sample'] = np.where(
    external_data['clear_text'].isin(sample['clear_text'].values),
    1,
    0
)

In [134]:
external_data.to_csv('../data/external_with_samplelabel.csv', index=False)

In [135]:
external_dataхъ

,class,link,path,text,clear_text,clear_text_in_sample
0,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-registracziyu-ts-v-gibd...,Доверенность\n\n\n\nг. Ижевск\n\nДата выдачи: ...,доверенность г. ижевск дата выдачи: третье сен...,1
1,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-zaverenie-kopij-dokumentov,"ООО ""Артемида""\n\n426000, г. Ижевск, ул. Сосно...","ооо артемида г. ижевск, ул. сосновая, д. а, те...",1
2,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-denezhnyh-sr...,"Общество с ограниченной ответственностью ""Сигм...",общество ограниченной ответственностью сигма г...,1
3,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-predstavlenie-interesov...,Доверенность\n\n\n\n\n\n\n\nг. Ижевск\n\nДвадц...,доверенность г. ижевск двадцать пятое сентября...,1
4,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-trudovoj-kni...,Доверенность\n\n\n\nг. Ижевск\n\n\n\nтринадцат...,доверенность г. ижевск тринадцатое марта две т...,1
...,...,...,...,...,...,...
1458,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-o-smene-yuridicheskogo-adresa...,ООО «Улыбка»\n\n\n\nРешение №6\n\n\n\nЕдинстве...,ооо улыбка решение единственного участника сме...,1
1459,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-uchreditelya-o-smene-direktor...,Решение №33\n\nЕдинственного участника ООО «Зе...,решение единственного участника ооо зеленая ми...,1
1460,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-resheniye-ychrediteley-ob-odobrenii-kr...,РЕШЕНИЕ № 123\n\nвнеочередного Общего собрани...,решение внеочередного общего собрания участник...,1
1461,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-o-smene-naimenovaniya-ooo,РЕШЕНИЕ № 07\n\nединственного Участника Общест...,решение единственного участника общества огран...,1
